<a href="https://colab.research.google.com/github/thistle0420/youtubeLiveChat/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.0 MB/s eta 0:00:00


In [70]:
# YouTube動画のURL
URL = 'https://www.youtube.com/watch?v=4_7tIEYyQKQ'

# 動画ID
video_id = URL.split('=')[1].split('&')[0]

# GoogleSpreadsheetのID
KEY = '1dhkLF2naw3eilWnLi-kZtrd6xT-L-YoIMPQ1JxtWuzA'

In [69]:
from yt_dlp import YoutubeDL

ydl_video_opts = {
    'outtmpl' : '%(id)s'+'_.mp4',
    'format' : 'best',
    'writesubtitles' : True,
    'skip_download' : True
}

with YoutubeDL(ydl_video_opts) as ydl:
    result = ydl.download([URL])

[youtube] Extracting URL: https://www.youtube.com/watch?v=4_7tIEYyQKQ
[youtube] 4_7tIEYyQKQ: Downloading webpage
[youtube] 4_7tIEYyQKQ: Downloading ios player API JSON
[youtube] 4_7tIEYyQKQ: Downloading android player API JSON
[youtube] 4_7tIEYyQKQ: Downloading m3u8 information
[info] 4_7tIEYyQKQ: Downloading subtitles: live_chat
[info] 4_7tIEYyQKQ: Downloading 1 format(s): 22
Deleting existing file 4_7tIEYyQKQ_.live_chat.json
[info] Writing video subtitles to: 4_7tIEYyQKQ_.live_chat.json
[youtube_live_chat] Downloading live chat
[youtube_live_chat] Total fragments: unknown (live)
[download] Destination: 4_7tIEYyQKQ_.live_chat.json
[download] 100% of   32.31MiB in 00:01:21 at 406.44KiB/s   


In [74]:
import json
import pandas as pd

def getTextMessage(item):
      try:
        timestampText = item["liveChatTextMessageRenderer"]["timestampText"]["simpleText"]
      except Exception:
        timestampText = ""

      try:
        author = item["liveChatTextMessageRenderer"]["authorName"]["simpleText"]
      except Exception:
        author = ""

      purchase_amount = ""
      message = ""

      try:
        message_items = item["liveChatTextMessageRenderer"]["message"]["runs"]
        for element in message_items:
          if "text" in element:
            message += element["text"]
          elif "emoji" in element:
            message += element["emoji"]["image"]["accessibility"]["accessibilityData"]["label"]
      except Exception:
        message = ""

      return [timestampText, author, purchase_amount, message]

def getPaidMessage(item):
      try:
        timestampText = item["liveChatPaidMessageRenderer"]["timestampText"]["simpleText"]
      except Exception:
        timestampText = ""

      try:
        author = item["liveChatPaidMessageRenderer"]["authorName"]["simpleText"]
      except Exception:
        author = ""

      try:
        purchase_amount = item["liveChatPaidMessageRenderer"]["purchaseAmountText"]["simpleText"]
      except Exception:
        purchase_amount = ""

      message = ""

      try:
        if "message" in item["liveChatPaidMessageRenderer"]:
            message_items = item["liveChatPaidMessageRenderer"]["message"]["runs"]
            for element in message_items:
              if "text" in element:
                message += element["text"]
              elif "emoji" in element:
                message += element["emoji"]["image"]["accessibility"]["accessibilityData"]["label"]
      except Exception:
        message = ""

      return [timestampText, author, purchase_amount, message]

input_file_name = f'{video_id}_.live_chat.json'
output = [['オフセット','タイムスタンプ','ユーザ名','スパチャ','コメント']]

with open(input_file_name, 'r', encoding='utf-8') as infile:
  lines = infile.readlines()
  for line in lines:
    data = json.loads(line)
    action = data["replayChatItemAction"]["actions"][0]
    videoOffsetTimeMsec = data["replayChatItemAction"]["videoOffsetTimeMsec"]

    if "addChatItemAction" not in action:
      continue

    item = action["addChatItemAction"]["item"]

    # 通常メッセージ
    if "liveChatTextMessageRenderer" in item:
      liveChat = getTextMessage(item)

    # 有料メッセージ処理
    elif "liveChatPaidMessageRenderer" in item:
      liveChat = getPaidMessage(item)

    else:
      continue

    output.append([videoOffsetTimeMsec] + liveChat)

{"clickTrackingParams": "CAEQl98BIhMIo92XicDSgQMVhcNgCh1T5gN_", "replayChatItemAction": {"actions": [{"clickTrackingParams": "CAEQl98BIhMIo92XicDSgQMVhcNgCh1T5gN_", "addChatItemAction": {"item": {"liveChatViewerEngagementMessageRenderer": {"id": "CiMKIVJFUExBWV9WRU0yMDIzLzA5LzMwLTA3OjAzOjI0LjQ4Nw%3D%3D", "timestampUsec": "1696082604487389", "icon": {"iconType": "YOUTUBE_ROUND"}, "message": {"runs": [{"text": "Live chat replay is on. Messages that appeared when the stream was live will show up here."}]}, "trackingParams": "CAEQl98BIhMIo92XicDSgQMVhcNgCh1T5gN_"}}}}], "videoOffsetTimeMsec": "0"}}

{"clickTrackingParams": "CAEQl98BIhMIo92XicDSgQMVhcNgCh1T5gN_", "replayChatItemAction": {"actions": [{"clickTrackingParams": "CAEQl98BIhMIo92XicDSgQMVhcNgCh1T5gN_", "addChatItemAction": {"item": {"liveChatMembershipItemRenderer": {"id": "ChwKGkNKUzBvUEw4ejRFREZmN0h3Z1FkSnBBQzdB", "timestampUsec": "1695995851712540", "timestampText": {"simpleText": "-4:39"}, "authorExternalChannelId": "UCz_MbjBcN

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




{"replayChatItemAction": {"actions": [{"addChatItemAction": {"item": {"liveChatTextMessageRenderer": {"message": {"runs": [{"text": "いつもの茶番かと思った"}]}, "authorName": {"simpleText": "ほしの"}, "authorPhoto": {"thumbnails": [{"url": "https://yt4.ggpht.com/sN5JWNcx_FfVthmESyAoEOFXUd3UnXzfrEbsZLBjARtFbVebYTX67IQy_Lfanz0Q7LSYv45sd0M=s32-c-k-c0x00ffffff-no-rj", "width": 32, "height": 32}, {"url": "https://yt4.ggpht.com/sN5JWNcx_FfVthmESyAoEOFXUd3UnXzfrEbsZLBjARtFbVebYTX67IQy_Lfanz0Q7LSYv45sd0M=s64-c-k-c0x00ffffff-no-rj", "width": 64, "height": 64}]}, "contextMenuEndpoint": {"commandMetadata": {"webCommandMetadata": {"ignoreNavigation": true}}, "liveChatItemContextMenuEndpoint": {"params": "Q2g0S0hBb2FRMGx0TFRoaE5rOHdTVVZFUm1VMFNISlJXV1EyTjAxRU5GRWFLU29uQ2hoVlEzUXpNR3BLWjBOb1REaHhaVlE1VmxCaFpHbGtVM2NTQ3pSZk4zUkpSVmw1VVV0UklBRW9BVElhQ2hoVlEzcGZTMUp6UlMxcFZuaElWVnBZU21oYVZVMUpjV2M0QWtnQlVBRSUzRA=="}}, "id": "ChwKGkNJbS04YTZPMElFREZlNEhyUVlkNjdNRDRR", "timestampUsec": "1696000542279194", "authorExte

In [75]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.open_by_key(KEY)
try:
  sh.add_worksheet(video_id, rows=30, cols=20)
except Exception as e :
  sh.worksheet(video_id).clear()

sh.values_append(video_id, {"valueInputOption": "USER_ENTERED"}, {"values": output})

{'spreadsheetId': '1dhkLF2naw3eilWnLi-kZtrd6xT-L-YoIMPQ1JxtWuzA',
 'updates': {'spreadsheetId': '1dhkLF2naw3eilWnLi-kZtrd6xT-L-YoIMPQ1JxtWuzA',
  'updatedRange': "'4_7tIEYyQKQ'!A1:E19822",
  'updatedRows': 19822,
  'updatedColumns': 5,
  'updatedCells': 99110}}